# Quasi-Markovian Approximation of the Generalized Langevin Equation (QM-GLE)

**Feiyu Zhu**


## Simplified QM-GLE 

After applying an orthogonal transformation to diagonalize the positive definite matrix in the memory kernel, we can simplify the original QM-GLE to the following SDE:

$$
\begin{aligned}
\ud X(t) &= V(t) \ud t \\
\ud V(t) &= -U'(X(t)) \ud t + \sum_{i=1}^m \mu_i Z_i(t)  \ud t \\
\ud Z(t) &= -\left(\mu_i V(t) + \rho_i Z_i(t) \right) \ud t + \sqrt{2\rho_i}/\beta \ud W_i(t), \qquad i = 1,\ldots,m
\end{aligned}
$$

where $X(t)$ is the observed (continuous-time) process whereas $V(t), Z(t)$ are latent, $U(\cdot)$ is the potential energy function ($U'$ is its first-order derivative), $\beta$, $\mu_i$, $\rho_i$ are unknown (positive) parameters, $\WW(t) = (W_1(t), \ldots, W_m(t))$ is an $m$-dimensional Brownian motion.

## Euler-Maruyama Discretization

$$
\begin{aligned}
X_n - X_{n-1} &= V_{n-1} \dt \\
V_{n} - V_{n-1} &= U'(X_{n-1})\dt + \sum_{i=1}^m \mu_i Z_{i,n-1} \dt \\
Z_{i,n} - Z_{i,n-1} &= - \left(\mu_i V_{n-1} + \rho_i Z_{i,n-1} \right)\dt + \sqrt{2\rho_i}/\beta \Delta W_{i,n-1}
\end{aligned}
$$

where $\Delta W_{i,n} = W_{i,n} - W_{i, n-1} \iid \N(0, \dt)$, $i=1,\ldots,m$, $m \ge 1$ is predetermined.

## Idea 1: Bridge Proposal 

Let $\WW_{n} = (W_{1,n}, \ldots, W_{m,n})$.

$$
\begin{aligned}
\Delta \WW_{n-1} &\sim \N(\bz, \dt \II)\\
Z_{i,n} \mid \WW & \sim \N(W_{i,n-1} + \mmu_{Z|W}, 2\rho_i/\beta^2 \dt \II), \quad \mmu_{Z|W} = - \left(\mu_i V_{n-1} + \rho_i Z_{i,n-1} \right)\dt,  \\
V_n \mid X_{n-1}, Z_{n-1}, \WW &\sim \N(V_{n-1} + \mu_{V|Z}, \bz), \mu_{V|Z} = U'(X_{n-1})\dt + \sum_{i=1}^m \mu_i Z_{i,n-1} \dt \\
X_n \mid V_{n-1}, Z_{n-1}, \WW & \sim \N(X_{n-1} + \mu_{X|V}, \bz), \mu_{X|V} = \dt \mu_{V|Z}
\end{aligned}
$$

## Idea 2: Simulation based on Integrated Wiener Process ($m=1$)



The reason why we try to apply the integrated Wiener process is that we want to introduce more randomness to the first and second equations (which are deterministic) in the QM-GLE SDE.

Draw the initial values $Z_0, V_0, X_0$ from some priors.

Generate a multivariate normal random vector $\WW_n = (\Delta W^{(2)}_n, \Delta W^{(1)}_n, \Delta W_n)$ 

$$
\WW_n \sim \N\left(\bm{0}, \SSi \right), \qquad 
\SSi = \begin{pmatrix}
  \frac{(\Delta t)^5}{20}  & \frac{(\Delta t)^4}{8} & \frac{(\Delta t)^3}{6} \\
  \frac{(\Delta t)^4}{8}   & \frac{(\Delta t)^3}{3}  & \frac{(\Delta t)^2}{2} \\
  \frac{(\Delta t)^3}{6}   & \frac{(\Delta t)^2}{2}  & \Delta t
\end{pmatrix}
$$

Given samples at time $n\Delta t$ ($n=0,1,\ldots$), we can generate $Z_{n+1}, V_{n+1}, X_{n+1}$ as follows

$$
\begin{aligned}
  Z_{n+1} &= Z_n - \left(\mu V_n + \rho Z_n \right)\Delta t + \sigma \Delta W_n \\
  V_{n+1} &= V_n - U'(X_n)\Delta t + \mu Z_n\Delta t - \mu \left(\mu V_n + \rho Z_n\right)\frac{(\Delta t)^2}{2} + \mu\sigma\Delta W^{(1)}_n \\
  X_{n+1} &= X_n - U'(X_n)\frac{(\Delta t)^2}{2} + \mu Z_n\frac{(\Delta t)^2}{2} - \mu \left(\mu V_n + \rho Z_n \right)\frac{(\Delta t)^3}{6} + \mu\sigma \Delta W^{(2)}_n
\end{aligned}
$$

We only need to calculate $Z_n, V_n$ for $n=1,\ldots,N-1$ to obtain the sequence $X_1, \ldots, X_N$.
